In [99]:
import pickle
import torch
import io
import warnings
warnings.filterwarnings('ignore')

model_args = (0.05, 100, 0.05, 16, 0.0, 5) # learning rate - epochs - lambda - batch size - weight decay - num folds
FOLD = 4

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

with open('matched_patients.pickle', 'rb') as f:
    df = pickle.load(f)

with open(f'ids_pidn_fold_{FOLD}_{model_args}.pickle', 'rb') as f:
    ids = pickle.load(f)

ids = [int(x) for x in ids.reshape((-1, 1))]

with open(f'final_preds_fold_{FOLD}_{model_args}.pickle', 'rb') as f:
    preds = CPU_Unpickler(f).load()
    preds = torch.vstack([torch.tensor(p) for p in preds])


In [100]:
df = df.set_index('id')
df = df.drop(columns={'filename', 'image'})
df = df.loc[ids]

In [103]:
df['pred'] = list(preds.numpy().round(4))
df['correct'] = [(df['pred'].values[i].round() == df['label'].values[i]).all() for i in range(len(df))]
df[~df['npz'].isna()].head(10)

,age,gender,npz,label,pred,correct
id,,,,,,
8108,0.232309,0,0.496769,"[0, 0]","[0.4319, 0.2975]",True
19427,1.436490,1,-1.478889,"[1, 0]","[0.5968, 0.3802]",True
13288,-0.369781,0,-0.898105,"[0, 0]","[0.7767, 0.6281]",False
9183,-0.670826,1,0.711343,"[0, 0]","[0.9117, 0.3416]",False
28153,-1.875007,1,-1.398412,"[1, 0]","[0.5844, 0.3578]",True
26854,-0.520304,0,-0.324846,"[0, 0]","[0.7795, 0.3162]",False
22713,-0.369781,1,-0.370222,"[1, 0]","[0.7361, 0.4609]",True
26942,-1.423439,1,-0.906765,"[1, 0]","[0.6092, 0.6259]",False
21979,-0.670826,1,-1.285111,"[1, 0]","[0.3822, 0.2873]",False
